# NeuralNet Multi-Layer Perceptron!

In [8]:
# Activation Function: ReLU
# Loss Function: Cross-Entropy Error

def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T 

    x = x - np.max(x) # 오버플로 대책
    return np.exp(x) / np.sum(np.exp(x))

In [9]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    # 훈련 데이터가 원-핫 벡터라면 정답 레이블의 인덱스로 반환
    if t.size == y.size:
        t = t.argmax(axis=1)
             
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

In [10]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
    
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        return dx

In [11]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        return dx

In [12]:
class Relu:
    def __init__(self):
        self.mask = None
    
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx

### TwoLayer

In [13]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from collections import OrderedDict

# TwoLayer
class NeuralNetMLP:
    
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        self.lastlayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
    
    # x: 입력 데이터, t: 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastlayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1: t = np.argmax(t, axis=1)
            
        accuracy = np.sum(y == t) / float(x.shape[0])
        
        return accuracy
    
    def gradient(self, x, t):
        self.loss(x, t)
        
        dout = 1
        dout = self.lastlayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
        
        # 결과 저장
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        
        return grads

In [14]:
from tqdm.auto import tqdm
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

# load data
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = NeuralNetMLP(input_size=784, hidden_size=512, output_size=10)

epochs = 100
idx_arr = np.arange(len(x_train))
batch_size = 1000
learning_rate = 0.1


for epoch_i in range(epochs):
    train_loss_list = []
    np.random.shuffle(idx_arr)
    for idx in idx_arr.reshape(-1, batch_size):
        
        x_batch, t_batch = x_train[idx], t_train[idx]
        grad = network.gradient(x_batch, t_batch)
        
        for key in ('W1', 'b1', 'W2', 'b2'):
            network.params[key] -= learning_rate * grad[key]
            
        loss = network.loss(x_batch, t_batch)
        train_loss_list.append(loss)
    
    train_acc = network.accuracy(x_train, t_train)
    test_acc = network.accuracy(x_test, t_test)
    print('Epoch : {} / Loss : {:.4f} / Train Acc : {:.4f}% / Test Acc : {:.4f}%'.format(epoch_i, np.mean(train_loss_list), train_acc*100, test_acc*100))

Epoch : 0 / Loss : 1.9022 / Train Acc : 75.2900% / Test Acc : 76.1000%
Epoch : 1 / Loss : 0.8326 / Train Acc : 84.8783% / Test Acc : 85.4400%
Epoch : 2 / Loss : 0.5356 / Train Acc : 87.4717% / Test Acc : 88.1700%
Epoch : 3 / Loss : 0.4400 / Train Acc : 88.7217% / Test Acc : 89.2700%
Epoch : 4 / Loss : 0.3932 / Train Acc : 89.5167% / Test Acc : 90.0100%
Epoch : 5 / Loss : 0.3648 / Train Acc : 89.9583% / Test Acc : 90.3700%
Epoch : 6 / Loss : 0.3449 / Train Acc : 90.4217% / Test Acc : 90.7900%
Epoch : 7 / Loss : 0.3299 / Train Acc : 90.7733% / Test Acc : 91.2000%
Epoch : 8 / Loss : 0.3176 / Train Acc : 91.0567% / Test Acc : 91.4400%
Epoch : 9 / Loss : 0.3071 / Train Acc : 91.2633% / Test Acc : 91.6900%
Epoch : 10 / Loss : 0.2976 / Train Acc : 91.6400% / Test Acc : 91.9100%
Epoch : 11 / Loss : 0.2888 / Train Acc : 91.7817% / Test Acc : 92.1100%
Epoch : 12 / Loss : 0.2813 / Train Acc : 92.1133% / Test Acc : 92.2800%
Epoch : 13 / Loss : 0.2738 / Train Acc : 92.2667% / Test Acc : 92.4800%
Ep

### ThreeLayer

In [15]:
# ThreeLayer
class NeuralNetMLP:
    
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size, weight_init_std=0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size1)
        self.params['b1'] = np.zeros(hidden_size1)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size1, hidden_size2)
        self.params['b2'] = np.zeros(hidden_size2)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size2, output_size)
        self.params['b3'] = np.zeros(output_size)
        
        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine3'] = Affine(self.params['W3'], self.params['b3'])
        self.lastlayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
    
    # x: 입력 데이터, t: 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastlayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1: t = np.argmax(t, axis=1)
            
        accuracy = np.sum(y == t) / float(x.shape[0])
        
        return accuracy
        
    def gradient(self, x, t):
        self.loss(x, t)
        
        dout = 1
        dout = self.lastlayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
        
        # 결과 저장
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        grads['W3'] = self.layers['Affine3'].dW
        grads['b3'] = self.layers['Affine3'].db
        
        return grads

In [16]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

# load data
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = NeuralNetMLP(input_size=784, hidden_size1=512, hidden_size2=512, output_size=10)

epochs = 100

idx_arr = np.arange(len(x_train))

batch_size = 1000
learning_rate = 0.1

for epoch_i in range(epochs):
    train_loss_list = []
    np.random.shuffle(idx_arr)
    for idx in idx_arr.reshape(-1, batch_size):
        
        x_batch, t_batch = x_train[idx], t_train[idx]
        grad = network.gradient(x_batch, t_batch)
        
        for key in ('W1', 'b1', 'W2', 'b2', 'W3', 'b3'):
            network.params[key] -= learning_rate * grad[key]
            
        loss = network.loss(x_batch, t_batch)
        train_loss_list.append(loss)
    
    train_acc = network.accuracy(x_train, t_train)
    test_acc = network.accuracy(x_test, t_test)
    
    print('Epoch : {} / Loss : {:.4f} / Train Acc : {:.4f}% / Test Acc : {:.4f}%'.format(epoch_i, np.mean(train_loss_list),
                                                                                         train_acc*100, test_acc*100))

Epoch : 0 / Loss : 2.2965 / Train Acc : 20.7950% / Test Acc : 20.8600%
Epoch : 1 / Loss : 2.2490 / Train Acc : 40.7033% / Test Acc : 41.4700%
Epoch : 2 / Loss : 1.6983 / Train Acc : 67.9200% / Test Acc : 68.4700%
Epoch : 3 / Loss : 0.8485 / Train Acc : 79.8617% / Test Acc : 80.6800%
Epoch : 4 / Loss : 0.5938 / Train Acc : 84.5700% / Test Acc : 84.7100%
Epoch : 5 / Loss : 0.4820 / Train Acc : 86.7650% / Test Acc : 87.2100%
Epoch : 6 / Loss : 0.4232 / Train Acc : 88.1600% / Test Acc : 88.2200%
Epoch : 7 / Loss : 0.3866 / Train Acc : 89.0167% / Test Acc : 88.9000%
Epoch : 8 / Loss : 0.3615 / Train Acc : 89.7683% / Test Acc : 89.8200%
Epoch : 9 / Loss : 0.3429 / Train Acc : 90.2100% / Test Acc : 90.4200%
Epoch : 10 / Loss : 0.3275 / Train Acc : 90.6600% / Test Acc : 90.8200%
Epoch : 11 / Loss : 0.3136 / Train Acc : 91.0917% / Test Acc : 91.0600%
Epoch : 12 / Loss : 0.3012 / Train Acc : 91.4400% / Test Acc : 91.4600%
Epoch : 13 / Loss : 0.2894 / Train Acc : 91.5433% / Test Acc : 91.5000%
Ep

### FiveLayer

In [20]:
# FiveLayer
class NeuralNetMLP:
    
    def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, hidden_size4, 
                 output_size, weight_init_std=0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size1)
        self.params['b1'] = np.zeros(hidden_size1)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size1, hidden_size2)
        self.params['b2'] = np.zeros(hidden_size2)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size2, hidden_size3)
        self.params['b3'] = np.zeros(hidden_size3)
        self.params['W4'] = weight_init_std * np.random.randn(hidden_size3, hidden_size4)
        self.params['b4'] = np.zeros(hidden_size4)
        self.params['W5'] = weight_init_std * np.random.randn(hidden_size4, output_size)
        self.params['b5'] = np.zeros(output_size)
        
        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine3'] = Affine(self.params['W3'], self.params['b3'])
        self.layers['Relu3'] = Relu()
        self.layers['Affine4'] = Affine(self.params['W4'], self.params['b4'])
        self.layers['Relu4'] = Relu()
        self.layers['Affine5'] = Affine(self.params['W5'], self.params['b5'])
        self.lastlayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
    
    # x: 입력 데이터, t: 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastlayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1: t = np.argmax(t, axis=1)
            
        accuracy = np.sum(y == t) / float(x.shape[0])
        
        return accuracy
        
    def gradient(self, x, t):
        self.loss(x, t)
        
        dout = 1
        dout = self.lastlayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
        
        # 결과 저장
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        grads['W3'] = self.layers['Affine3'].dW
        grads['b3'] = self.layers['Affine3'].db
        grads['W4'] = self.layers['Affine4'].dW
        grads['b4'] = self.layers['Affine4'].db
        grads['W5'] = self.layers['Affine5'].dW
        grads['b5'] = self.layers['Affine5'].db
        
        return grads

In [21]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
from collections import OrderedDict

# load data
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = NeuralNetMLP(input_size=784, hidden_size1=512, hidden_size2=512, hidden_size3=512, hidden_size4=512,
                       output_size=10)

epochs = 100

idx_arr = np.arange(len(x_train))

batch_size = 1000
learning_rate = 0.1

for epoch_i in range(epochs):
    train_loss_list = []
    np.random.shuffle(idx_arr)
    for idx in idx_arr.reshape(-1, batch_size):
        
        x_batch, t_batch = x_train[idx], t_train[idx]
        grad = network.gradient(x_batch, t_batch)
        
        for key in ('W1', 'b1', 'W2', 'b2', 'W3', 'b3', 'W4', 'b4', 'W5', 'b5'):
            network.params[key] -= learning_rate * grad[key]
            
        loss = network.loss(x_batch, t_batch)
        train_loss_list.append(loss)
    
    train_acc = network.accuracy(x_train, t_train)
    test_acc = network.accuracy(x_test, t_test)
    
    print('Epoch : {} / Loss : {:.4f} / Train Acc : {:.4f}% / Test Acc : {:.4f}%'.format(epoch_i, np.mean(train_loss_list),
                                                                                         train_acc*100, test_acc*100))

Epoch : 0 / Loss : 2.3019 / Train Acc : 11.2367% / Test Acc : 11.3500%
Epoch : 1 / Loss : 2.3013 / Train Acc : 11.2367% / Test Acc : 11.3500%
Epoch : 2 / Loss : 2.3011 / Train Acc : 11.2367% / Test Acc : 11.3500%
Epoch : 3 / Loss : 2.3010 / Train Acc : 11.2367% / Test Acc : 11.3500%
Epoch : 4 / Loss : 2.3010 / Train Acc : 11.2367% / Test Acc : 11.3500%
Epoch : 5 / Loss : 2.3010 / Train Acc : 11.2367% / Test Acc : 11.3500%
Epoch : 6 / Loss : 2.3009 / Train Acc : 11.2367% / Test Acc : 11.3500%
Epoch : 7 / Loss : 2.3009 / Train Acc : 11.2367% / Test Acc : 11.3500%
Epoch : 8 / Loss : 2.3009 / Train Acc : 11.2367% / Test Acc : 11.3500%
Epoch : 9 / Loss : 2.3008 / Train Acc : 11.2367% / Test Acc : 11.3500%
Epoch : 10 / Loss : 2.3008 / Train Acc : 11.2367% / Test Acc : 11.3500%
Epoch : 11 / Loss : 2.3007 / Train Acc : 11.2367% / Test Acc : 11.3500%
Epoch : 12 / Loss : 2.3007 / Train Acc : 11.2367% / Test Acc : 11.3500%
Epoch : 13 / Loss : 2.3006 / Train Acc : 11.2367% / Test Acc : 11.3500%
Ep